In [25]:
import numpy as np
from itertools import combinations_with_replacement
from liblinear.liblinearutil import *

In [26]:
help(train)

Help on function train in module liblinear.liblinearutil:

train(arg1, arg2=None, arg3=None)
    train(y, x [, options]) -> model | ACC
    
    y: a list/tuple/ndarray of l true labels (type must be int/double).
    
    x: 1. a list/tuple of l training instances. Feature vector of
          each training instance is a list/tuple or dictionary.
    
       2. an l * n numpy ndarray or scipy spmatrix (n: number of features).
    
    train(prob [, options]) -> model | ACC
    train(prob, param) -> model | ACC
    
    Train a model from data (y, x) or a problem prob using
    'options' or a parameter param.
    
    If '-v' is specified in 'options' (i.e., cross validation)
    either accuracy (ACC) or mean-squared error (MSE) is returned.
    
    options:
        -s type : set type of solver (default 1)
          for multi-class classification
             0 -- L2-regularized logistic regression (primal)
             1 -- L2-regularized L2-loss support vector classification (dual)
  

In [27]:
def polynomial_transformation(x, order):
    n, index = x.shape[0], list(range(x.shape[1]))
    new_x = np.ones((x.shape[0], 1))
    for o in range(1, order+1):
        idx_set = np.array(list(combinations_with_replacement(index, o)))
        for set in idx_set:
            new_col = np.ones((x.shape[0], 1))
            for i in set:
                new_col *= np.reshape(x[:,i], (n, 1))
            new_x = np.hstack((new_x, new_col))
    return new_x


In [28]:
"""
load data and preprocess
"""
# read data
with open('hw4_train.dat', 'rb') as f:  
    training_data = np.array([np.float64(i.split()) for i in f.readlines()])
    
# turn x and y into numpy array
# x is the input feature vector space and y is the corresponding label
x = np.array(training_data[:,0:10])
y = np.reshape(np.array(list(map(int, training_data[:,10]))), (len(x), 1))
print(x.shape)
print(y.shape)

N = len(x)

with open('hw4_test.dat', 'rb') as f:  
    test_data = np.array([np.float64(i.split()) for i in f.readlines()])
    
# turn x and y into numpy array
# x is the input feature vector space and y is the corresponding label
x_test = np.array(test_data[:,0:10])
y_test = np.reshape(np.array(list(map(int, test_data[:,10]))), (len(x_test), 1))
print(x_test.shape)
print(y_test.shape)

N = len(x)

(200, 10)
(200, 1)
(500, 10)
(500, 1)


In [29]:
"""
polynomial transformation
"""
order = 4
x_transform = polynomial_transformation(x, order)
x_test_transform = polynomial_transformation(x_test, order)

# for i in range(x_transform.shape[0]):
#     print(x_transform[i])
    
print(x_transform.shape)
print(x_test_transform.shape)

y = y.flatten()
y_test = y_test.flatten()
print(y.shape)

(200, 1001)
(500, 1001)
(200,)


In [30]:
"""
P12
"""

lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)
prob = problem(y, x_transform)
E_list = []
for l in lamb:
    parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
    # print(1/(2*(10**l)))
    model = train(prob, parm)
    p_label, p_acc, p_val = predict(y_test, x_test_transform, model)
    E_list.append(np.mean(p_label != y_test))

best_e = 1.1
best_lambda = -np.inf
for idx, e in enumerate(E_list):
    if e < best_e:
        best_e = e
        best_lambda = lamb[idx]
    elif e == best_e and lamb[idx] > best_lambda:
        best_lambda = lamb[idx]
        best_idx = idx


Accuracy = 77.4% (387/500) (classification)
Accuracy = 82.2% (411/500) (classification)
Accuracy = 84.6% (423/500) (classification)
Accuracy = 85.8% (429/500) (classification)
Accuracy = 81.2% (406/500) (classification)


In [31]:
"""
p12 ans
"""
print(lamb)
print(E_list)        
print(f'best error = {best_e}, best lambda = {best_lambda}')

[-6. -3.  0.  3.  6.]
[0.226, 0.178, 0.154, 0.142, 0.188]
best error = 0.142, best lambda = 3.0


In [32]:
"""
P13
"""
lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)
prob = problem(y, x_transform)
E_list = []
for l in lamb:
    parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
    # print(1/(2*(10**l)))
    model = train(prob, parm)
    p_label, p_acc, p_val = predict(y, x_transform, model)
    E_list.append(np.mean(p_label != y))

best_e = 1.1
best_lambda = -np.inf
for idx, e in enumerate(E_list):
    if e < best_e:
        best_e = e
        best_lambda = lamb[idx]
    elif e == best_e and lamb[idx] > best_lambda:
        best_lambda = lamb[idx]
        best_idx = idx


Accuracy = 100% (200/200) (classification)
Accuracy = 100% (200/200) (classification)
Accuracy = 100% (200/200) (classification)
Accuracy = 96% (192/200) (classification)
Accuracy = 76% (152/200) (classification)


In [33]:
"""
p13 ans
"""
print(lamb)
print(E_list)
print(f'best error = {best_e}, best lambda = {best_lambda}')

[-6. -3.  0.  3.  6.]
[0.0, 0.0, 0.0, 0.04, 0.24]
best error = 0.0, best lambda = 0.0


In [ ]:
"""
P14
"""

lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)

lamb_cnt = {}
for l in lamb:
    lamb_cnt[l] = 0

for i in range(256):
    # generate random numbers between 0 to len(x_transform) without repeat, this step is equal to shuffle the data
    rng = np.random.default_rng()
    numbers = rng.choice(len(x_transform), size=len(x_transform), replace=False)

    # 120 tranining samples
    x_train = x_transform[numbers[:120]]
    y_train = y[numbers[:120]]
    # 80 evaluation samples
    x_eval = x_transform[numbers[120:]]
    y_eval = y[numbers[120:]]
    
    E_list = []
    
    prob = problem(y_train, x_train)
    for l in lamb:
        parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
        # print(1/(2*(10**l)))
        model = train(prob, parm)
        p_label, p_acc, p_val = predict(y_eval, x_eval, model)
        E_list.append(np.mean(p_label != y_eval))

    best_e = 1.1
    best_lambda = -np.inf
    for idx, e in enumerate(E_list):
        if e < best_e:
            best_e = e
            best_lambda = lamb[idx]
            best_idx = idx
        elif e == best_e and lamb[idx] > best_lambda:
            best_lambda = lamb[idx]
            best_idx = idx
            
    # print(Eout_list)
    # print(f'best error = {best_e}, best lambda = {best_lambda}')
    lamb_cnt[best_lambda] += 1
    
    # I delete the training output(Accuracy part), cuz its too lengthy and cannot be removed by specify -q

In [35]:
"""
p14 ans
"""
print(lamb_cnt)
max(lamb_cnt, key=lamb_cnt.get)

{-6.0: 1, -3.0: 20, 0.0: 77, 3.0: 157, 6.0: 1}


3.0

In [ ]:
"""
P15
"""

lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)

lamb_cnt = {}
for l in lamb:
    lamb_cnt[l] = 0
    
p15_ans = []
for i in range(256):
    # generate random numbers between 0 to len(x_transform) without repeat, this step is equal to shuffle the data
    rng = np.random.default_rng()
    numbers = rng.choice(len(x_transform), size=len(x_transform), replace=False)

    # 120 tranining samples
    x_train = x_transform[numbers[:120]]
    y_train = y[numbers[:120]]
    # 80 evaluation samples
    x_eval = x_transform[numbers[120:]]
    y_eval = y[numbers[120:]]
    
    E_list = []
    
    prob = problem(y_train, x_train)
    for l in lamb:
        parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
        # print(1/(2*(10**l)))
        model = train(prob, parm)
        p_label, p_acc, p_val = predict(y_eval, x_eval, model)
        E_list.append(np.mean(p_label != y_eval))

    best_e = 1.1
    best_lambda = -np.inf
    for idx, e in enumerate(E_list):
        if e < best_e:
            best_e = e
            best_lambda = lamb[idx]
            best_idx = idx
        elif e == best_e and lamb[idx] > best_lambda:
            best_lambda = lamb[idx]
            best_idx = idx

    prob = problem(y_train, x_train)
    parm = parameter(f'-s 0 -c {1/(2*(10**best_lambda))} -e 0.000001')
    model = train(prob, parm)
    p_label, p_acc, p_val = predict(y_test, x_test_transform, model)
    
    p15_ans.append(np.mean(p_label != y_test))
    # print(Eout_list)
    # print(f'best error = {best_e}, best lambda = {best_lambda}')
    lamb_cnt[best_lambda] += 1
    
    # I delete the training output(Accuracy part), cuz its too lengthy and cannot be removed by specify -q

In [37]:
"""
p15 ans
"""
print(np.mean(p15_ans))

0.167234375


In [ ]:
"""
P16
"""

lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)

lamb_cnt = {}
for l in lamb:
    lamb_cnt[l] = 0


p16_ans = []
for i in range(256):
    # generate random numbers between 0 to len(x_transform) without repeat, this step is equal to shuffle the data
    rng = np.random.default_rng()
    numbers = rng.choice(len(x_transform), size=len(x_transform), replace=False)

    # 120 tranining samples
    x_train = x_transform[numbers[:120]]
    y_train = y[numbers[:120]]
    # 80 evaluation samples
    x_eval = x_transform[numbers[120:]]
    y_eval = y[numbers[120:]]
    
    E_list = []
    
    prob = problem(y_train, x_train)
    for l in lamb:
        parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
        # print(1/(2*(10**l)))
        model = train(prob, parm)
        p_label, p_acc, p_val = predict(y_eval, x_eval, model)
        E_list.append(np.mean(p_label != y_eval))

    best_e = 1.1
    best_lambda = -np.inf
    for idx, e in enumerate(E_list):
        if e < best_e:
            best_e = e
            best_lambda = lamb[idx]
            best_idx = idx
        elif e == best_e and lamb[idx] > best_lambda:
            best_lambda = lamb[idx]
            best_idx = idx
    prob = problem(y, x_transform)
    parm = parameter(f'-s 0 -c {1/(2*(10**best_lambda))} -e 0.000001')
    model = train(prob, parm)
    p_label, p_acc, p_val = predict(y_test, x_test_transform, model)
    

    p16_ans.append(np.mean(p_label != y_test))
    lamb_cnt[best_lambda] += 1
    # I delete the training output(Accuracy part), cuz its too lengthy and cannot be removed by specify -q

In [39]:
"""
p16 ans
"""
print(np.mean(p16_ans))

0.14942968749999996


In [ ]:
"""
P17
"""
lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)
ans = []
for i in range(256):
    # generate random numbers between 0 to len(x_transform) without repeat, this step is equal to shuffle the data
    rng = np.random.default_rng()
    numbers = rng.choice(len(x_transform), size=len(x_transform), replace=False)
    
    fold_num = 5
    # slice into n-folds
    # length of each fold
    length = int(len(numbers)/fold_num) 
    folds = []
    for k in range(fold_num-1):
        folds += [numbers[k*length:(k+1)*length]]
    folds += [numbers[(fold_num-1)*length:len(numbers)]]
    E = []

    for l in lamb:
        E_list = []
        
        for j in range(fold_num):
            temp = folds.copy()
            # print(len(folds))
            valid_idx = np.array(temp.pop(j)).flatten()
            training_idx = np.array(temp).flatten()
            
            training_sample = x_transform[training_idx]
            training_y = y[training_idx]
            valid_sample = x_transform[valid_idx]
            valid_y = y[valid_idx]
        
            prob = problem(training_y, training_sample)
            
            parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
            model = train(prob, parm)
            p_label, p_acc, p_val = predict(valid_y, valid_sample, model)
            E_list.append(np.mean(p_label != valid_y))
        
        E.append(np.mean(E_list))
    ans.append(min(E))
    # I delete the training output(Accuracy part), cuz its too lengthy and cannot be removed by specify -q

In [41]:
"""
p17 ans
"""
print(np.mean(ans))

0.13078125000000002


In [42]:
"""
P18
"""
lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)
prob = problem(y, x_transform)
E_list = []
for l in lamb:
    parm = parameter(f'-s 6 -c {1/(10**l)} -e 0.000001')
    model = train(prob, parm)
    p_label, p_acc, p_val = predict(y_test, x_test_transform, model)
    E_list.append(np.mean(p_label != y_test))

best_e = 1.1
best_lambda = -np.inf
for idx, e in enumerate(E_list):
    if e < best_e:
        best_e = e
        best_lambda = lamb[idx]
    elif e == best_e and lamb[idx] > best_lambda:
        best_lambda = lamb[idx]
        best_idx = idx

Accuracy = 77.2% (386/500) (classification)
Accuracy = 84.4% (422/500) (classification)
Accuracy = 84.6% (423/500) (classification)
Accuracy = 68% (340/500) (classification)
Accuracy = 49.2% (246/500) (classification)


In [43]:
"""
p18 ans
"""
print(lamb)
print(E_list)        
print(f'best error = {best_e}, best lambda = {best_lambda}')

[-6. -3.  0.  3.  6.]
[0.228, 0.156, 0.154, 0.32, 0.508]
best error = 0.154, best lambda = 0.0


In [44]:
"""
P19
"""
# bast lambda from p18
prob = problem(y, x_transform)
parm = parameter(f'-s 6 -c {1/(10**best_lambda)} -e 0.000001')
model = train(prob, parm)

cnt = 0
for i in range(x_transform.shape[1]):
    if np.abs(model.w[i]) <= 10**-6:
        cnt += 1
        

In [45]:
"""
p19 ans
"""
cnt

960

In [46]:
"""
P20
"""
lamb = np.array([-6, -3, 0, 3, 6], dtype=np.float32)
prob = problem(y, x_transform)
E_list = []
for l in lamb:
    parm = parameter(f'-s 0 -c {1/(2*(10**l))} -e 0.000001')
    # print(1/(2*(10**l)))
    model = train(prob, parm)
    p_label, p_acc, p_val = predict(y_test, x_test_transform, model)
    E_list.append(np.mean(p_label != y_test))

best_e = 1.1
best_lambda = -np.inf
for idx, e in enumerate(E_list):
    if e < best_e:
        best_e = e
        best_lambda = lamb[idx]
    elif e == best_e and lamb[idx] > best_lambda:
        best_lambda = lamb[idx]
        best_idx = idx

prob = problem(y, x_transform)
parm = parameter(f'-s 0 -c {1/(2*(10**best_lambda))} -e 0.000001')
model = train(prob, parm)

cnt = 0
for i in range(x_transform.shape[1]):
    if np.abs(model.w[i]) <= 10**-6:
        cnt += 1
        

Accuracy = 77.4% (387/500) (classification)
Accuracy = 82.2% (411/500) (classification)
Accuracy = 84.6% (423/500) (classification)
Accuracy = 85.8% (429/500) (classification)
Accuracy = 81.2% (406/500) (classification)


In [47]:
"""
p20 ans
"""
cnt

1